In [196]:
import db_connection.ihub as ihub
import db_connection.alice as alice
import pandas as pd
import numpy as np
from scipy.stats import norm
import warnings

In [197]:
warnings.filterwarnings('ignore')
conn = ihub.iHub_engine('expedia').connection
conn2 = alice.alice_engine('hqlive').connection

In [198]:
try:
    with conn2.cursor() as cursor:
        query = """
            select
            hotel_id as Hotel_ID,
            d10_hotel_city_name as city,
            d15_hotel_name as hotel_name,
            hqlive.hotel.hotel_catg_id as stars,
            d25_hotel_location_latitude as lat,
            d26_hotel_location_longitude as lon,
            count(distinct order_id) as booking,
            sum(datediff(m139_offer_checkout_date, m138_offer_checkin_date)) * room_cnt as rns,
            round(sum(c1_selling_price),0) as gbv,
            round(sum(c1_selling_price),0) / sum(datediff(m139_offer_checkout_date, m138_offer_checkin_date)) as gbv_rns
            from bi_export.order
            left join hqlive.hotel
            on 1=1
            and bi_export.order.hotel_id = hqlive.hotel.id
            where order.hotel_id <> -1
            and datediff('2018-12-01', date(m01_order_datetime_gmt0)) < 30
            and hotel_status_id = 1
            and d22_inventory_source_code = 'HQ01'
            group by 1,2,3,4
            order by gbv desc
        """
        cursor.execute(query)
        conn.commit()
        df_from_query = pd.read_sql(query, conn2)
finally:
        conn2.close()

In [218]:
# dataframe:
df = pd.read_csv('C:\\Users\\mojiway\\Desktop\\recom\\recom_table_2.csv',index_col=None)

In [219]:
df.head()

,hotel_id,city,hotel_name,stars,lat,lon,booking,rns,gbv,gbv_rns
0,242495,Boston,The Godfrey Hotel Boston,4-star,42.354662,-71.061475,107,189,27007,143
1,53230,Marrakech,La Mamounia,5-star,31.621885,-7.997605,5,11,9068,824
2,311703,Nassau,Grand Hyatt At Baha Mar,4-star,25.070796,-77.396624,7,36,8416,234
3,8303,Bangkok,Shangri-La Hotel Bangkok,5-star,13.720692,100.513980,10,30,8151,272
4,8519,Bali,Four Seasons Resort Bali at Sayan,5-star,-8.495717,115.244937,5,18,7877,438


In [220]:
df['commission'] = 0.15

In [221]:
def get_hotel_id(url):
    response = urllib.request.urlopen(url)        
    response_text = response.read() 
    soup = BeautifulSoup(response_text)
    header = soup.find("div", {"class" : "card-header"}).extract()
    header2 = str(header)
    hotel_id = header2[header2.find(":") + 1:header2.find(")")]
    return hotel_id


In [222]:
hotel_link = "http://recommendation-simulation.com/hotels/242495"

In [223]:
get_hotel_id(hotel_link)

'242495'

In [224]:
city_input = 'Bangkok'

In [225]:
# data slicing based on the distribution:
def distribution_cut(col):
    mean, std = norm.fit(col)
    p0 = mean + 2 * std
    p1 = mean + std
    p2 = mean
    p3 = mean-std if std < mean else mean-0.4*std
    return p0, p1, p2, p3

In [226]:
def calculate_score(col_name):
    slices = distribution_cut(df[col_name])
    col_title = col_name + '_' + 'score' 
    df.loc[df[col_name] > slices[0], col_title] = 1000
    df.loc[(df[col_name] < slices[0]) & (df[col_name] > slices[1]) , col_title] = 750
    df.loc[(df[col_name] < slices[1]) & (df[col_name] > slices[2]) , col_title] = 500
    df.loc[(df[col_name] < slices[2]) & (df[col_name] > slices[3]) , col_title] = 250
    df.loc[df[col_name] < slices[3], col_title] = 100    

In [227]:
calculate_score('gbv_rns')

In [228]:
df['revenue'] = df['gbv_rns'] * df['commission']

In [229]:
df['city_score'] = np.where(df['city']== city_input, 1, 0)

In [230]:
calculate_score('booking')

In [231]:
calculate_score('gbv')

In [232]:
df['total_score'] = (2*df['gbv_rns_score'] + 2*df['booking_score'] \
             + 2*df['gbv_score'] + 5*df['revenue'])*df['city_score']

In [233]:
df.head()

,hotel_id,city,hotel_name,stars,lat,lon,booking,rns,gbv,gbv_rns,commission,gbv_rns_score,revenue,city_score,booking_score,gbv_score,total_score
0,242495,Boston,The Godfrey Hotel Boston,4-star,42.354662,-71.061475,107,189,27007,143,0.15,250.0,21.45,0,1000.0,1000.0,0.0
1,53230,Marrakech,La Mamounia,5-star,31.621885,-7.997605,5,11,9068,824,0.15,1000.0,123.60,0,500.0,1000.0,0.0
2,311703,Nassau,Grand Hyatt At Baha Mar,4-star,25.070796,-77.396624,7,36,8416,234,0.15,500.0,35.10,0,750.0,1000.0,0.0
3,8303,Bangkok,Shangri-La Hotel Bangkok,5-star,13.720692,100.513980,10,30,8151,272,0.15,500.0,40.80,1,1000.0,1000.0,5204.0
4,8519,Bali,Four Seasons Resort Bali at Sayan,5-star,-8.495717,115.244937,5,18,7877,438,0.15,1000.0,65.70,0,500.0,1000.0,0.0


In [242]:
df = df[df['total_score'] != 0].sort_values('total_score', ascending=False)

In [243]:
df

,hotel_id,city,hotel_name,stars,lat,lon,booking,rns,gbv,gbv_rns,commission,gbv_rns_score,revenue,city_score,booking_score,gbv_score,total_score
3,8303,Bangkok,Shangri-La Hotel Bangkok,5-star,13.720692,100.513980,10,30,8151,272,0.15,500.0,40.80,1,1000.0,1000.0,5204.00
15,786,Bangkok,Banyan Tree Bangkok,5-star,13.723229,100.539688,7,21,5219,249,0.15,500.0,37.35,1,750.0,1000.0,4686.75
80,8229,Bangkok,Novotel Bangkok on Siam Square,4-star,13.744694,100.534872,7,15,2257,150,0.15,250.0,22.50,1,750.0,750.0,3612.50
165,8070,Bangkok,Novotel Bangkok Platinum Pratunam,4-star,13.749696,100.540773,1,4,1384,346,0.15,750.0,51.90,1,250.0,500.0,3259.50
89,3572,Bangkok,The Landmark Bangkok,5-star,13.741121,100.553985,3,14,2119,151,0.15,250.0,22.65,1,500.0,750.0,3113.25
146,16202,Bangkok,Al Meroz Hotel Bangkok - The Leading Halal Hotel,4-star,13.745976,100.603878,1,6,1506,251,0.15,500.0,37.65,1,250.0,500.0,2688.25
520,8652,Bangkok,Radisson Blu Plaza Bangkok,5-star,13.734614,100.564088,2,2,413,207,0.15,500.0,31.05,1,500.0,250.0,2655.25
293,472,Bangkok,Centara Grand at CentralWorld,5-star,13.747829,100.538684,3,5,820,164,0.15,250.0,24.60,1,500.0,500.0,2623.00
290,8468,Bangkok,Aloft Bangkok Sukhumvit 11,4-star,13.744356,100.556519,3,6,830,138,0.15,250.0,20.70,1,500.0,500.0,2603.50
225,8274,Bangkok,Grande Centre Point Ratchadamri,5-star,13.741604,100.541049,2,8,1055,132,0.15,250.0,19.80,1,500.0,500.0,2599.00


In [244]:
df['hotel_id']

3        8303
15        786
80       8229
165      8070
89       3572
146     16202
520      8652
293       472
290      8468
225      8274
162      8455
572      8328
800      8258
818       788
843      8660
845     12434
374        33
645      8297
432       212
633      3562
739      7635
352      3387
484      2665
984     16220
751     18491
712       358
686      3558
430      8405
824     11964
477      8341
1235    21846
933      8347
1069     8656
1070     8602
1082     8670
1093    12380
1151    12608
1153       27
1194    12386
1195     5730
1202    12175
1054       58
1248     2909
1125    14293
1147    17368
1257    21845
1261    20880
1267    13348
1271     7611
1273     8644
1277    12154
1282     3566
1244    21538
1284    27157
1287    17971
1288      417
Name: hotel_id, dtype: int64